In [ ]:
def classify(row):
    avg_runs = row['cumulative_derived_Batting Avg']
    batting_sr = row['cumulative_derived_Batting Strike Rate']
    wickets = row['cumulative_Wickets_sum']
    matches_played = row['Total_matches_played_sum']
    wickets_per_match = wickets / matches_played if matches_played > 0 else 0
    bowling_avg = row['cumulative_derived_Bowling Avg']
    
    if avg_runs > 25 and batting_sr > 0.70 and wickets < 0.1 * matches_played:
        return 'Batter'
    elif wickets_per_match > 0.75 and bowling_avg < 35 and avg_runs < 25:
        return 'Bowler'
    elif avg_runs > 25 and wickets_per_match > 0.3 and batting_sr > 0.70:
        return 'All-Rounder'
    else:
        return 'Unclassified'

In [39]:
# Load the dataset
file_path = "/home/ai21btech11012/FIFS_dream11/src/data/processed/combined/10_ODI.csv"  # Change this to your actual file path
df = pd.read_csv(file_path)

# Initialize lists to store filtered data
player_ids = []
players = []
player_roles = []

previous_player_id = None

for index, row in df.iterrows():
    current_player_id = row['player_id']  # Assuming 'player_id' column exists
    player = row['player'] # Assuming 'player' column exists

    if current_player_id != previous_player_id:
        player_ids.append(current_player_id)
        players.append(player)
        player_roles.append(classify(row)) # Classify the row here

        previous_player_id = current_player_id

# Create a new DataFrame with only the required columns
new_df = pd.DataFrame({'player_id': player_ids, 'player': players, 'player_role': player_roles})

# Save the results
new_df.to_csv("classified_odi_players.csv", index=False)
print("Classification complete. Results saved to classified_odi_players.csv")

Classification complete. Results saved to classified_odi_players.csv


## TOP 5 players are batters

In [1]:
import json 

In [8]:
batter_dict = {}
def get_batters(json_path):
    # print(json_path)
    global batter_dict

    with open(json_path, "r") as file:
        cur_match = json.load(file)
    # print(cur_match['info']['gender'])
    if cur_match['info']['gender'] != 'male' :
        return None 
    
    for innings in range(len(cur_match['innings'])):
        curr_batters = set()
        for over in cur_match['innings'][innings]['overs']:
            for delivery in  over["deliveries"]:
                batter_name = delivery["batter"]
                batter_id = cur_match["info"]["registry"]["people"][batter_name]
                if batter_id not in curr_batters:
                    curr_batters.add(batter_id)
                if len(curr_batters)>=5:
                    break
            if len(curr_batters)>=5:
                    break
        for batter in curr_batters:
            if batter not in batter_dict:
                batter_dict[batter] = 1
            else:
                batter_dict[batter] += 1
        # print(curr_batters, "hi")
# get_batters("/home/ai21btech11012/FIFS_dream11/src/data/raw/cricsheet/all_json/1198240.json")

In [11]:
batter_dict

{'99cddc5e': 8,
 'e957b38f': 14,
 'cf494ffe': 105,
 '6ef13460': 77,
 '59ea70a4': 39,
 '9948e262': 61,
 'bf814547': 12,
 'f76d7466': 22,
 '6a26221c': 65,
 '9ffd1ac1': 39,
 'c500ec39': 7,
 'd8699ab7': 96,
 '7d415ea5': 188,
 '69762509': 71,
 '2b6e6dec': 117,
 'f76ffa81': 9,
 'f011dfa3': 1,
 'ea42ddb9': 61,
 '6e1b1ec0': 56,
 'd2f3829d': 11,
 'd0513f63': 49,
 'f842c2cf': 163,
 '508a1ea7': 67,
 '5748e866': 25,
 '48fd7349': 72,
 'ba607b88': 275,
 '1dc12ab9': 121,
 '1c914163': 174,
 '0a476045': 163,
 '4b57e452': 15,
 'cfad138c': 35,
 '896d78ad': 100,
 '2008fc75': 19,
 '5bdcdb72': 184,
 '7ed9fd56': 193,
 '740742ef': 234,
 'c03f1114': 55,
 '4329fbb5': 124,
 'db584dad': 197,
 '24ecc358': 10,
 '89f64c19': 55,
 'caf1b14c': 16,
 '2f8a9682': 34,
 'd1c36f5c': 109,
 'abb83e27': 84,
 'd2a6c0e6': 190,
 'a343262c': 155,
 '92aeac25': 65,
 'dcce6f09': 154,
 '3d8feaf8': 56,
 '30a45b23': 127,
 '12b610c2': 52,
 '53597be1': 159,
 'e7ba1233': 46,
 'afe57f7a': 104,
 'f32dbc84': 78,
 '31fbf891': 155,
 '7dc35884': 

In [10]:
import os
from joblib import Parallel, delayed
from collections import defaultdict

json_folder = "/home/ai21btech11012/FIFS_dream11/src/data/raw/cricsheet/all_json/"

all_files = os.listdir(json_folder)
all_paths = [os.path.join(json_folder, file) for file in all_files]

# Function to process a single file
def process_match(match_path):
    # bw_dict = get_bowler_stat_per_match(match_path, player_data_path)
    get_batters(match_path)
    return 

# Run parallel processing
# Parallel(n_jobs=-1)(delayed(process_match)(path) for path in all_paths)
for path in all_paths:
     process_match(path)


In [12]:
import pandas as pd

def correct_unclassified(df, batter_dict):
    """
    Corrects "Unclassified" player roles to "Batter" if the player_id is in batter_dict.

    Args:
        df: The Pandas DataFrame containing player data.
        batter_dict: A dictionary where keys are player_ids and values are relevant (can be anything, even None).  Used to check if a player is in the batter category.

    Returns:
        The modified Pandas DataFrame.
    """
    for idx, row in df.iterrows():
        if row["player_role"] == "Unclassified" and row["player_id"] in batter_dict:
            # Use .loc to modify the DataFrame correctly
            df.loc[idx, "player_role"] = "Batter"  # Corrected to "Batter"
    return df

# Example usage (replace with your actual data and dictionary):
df = pd.read_csv("classified_odi_players.csv")
# batter_dict = {1: None, 2: None, 5: None} # Example: Player IDs 1, 2, and 5 are batters

df = correct_unclassified(df, batter_dict)

print(df)
df.to_csv("corrected_classified_odi_players.csv", index=False) # Save the corrected DataFrame

     player_id           player   player_role
0     004c9e85        PJ Hughes        Batter
1     008589a4        CM Wright  Unclassified
2     00d69fa4        SO Tikolo        Batter
3     00ea847a       MA Agarwal        Batter
4     00ea9791  HP Tillakaratne        Batter
...        ...              ...           ...
1839  ff1ec94d          R Klein  Unclassified
1840  ff3f6fc1   Mohammad Haris        Batter
1841  ff6a7ab5    R Trumpelmann        Bowler
1842  ffb504b1      XM Marshall        Batter
1843  ffe699c0        CJ Jordan  Unclassified

[1844 rows x 3 columns]


In [6]:
df  = pd.read_csv("corrected_classified_odi_players.csv")
unclassified_count = (df["player_role"] == "Unclassified").sum()
unclassified_count

np.int64(506)

## PLayer classification using web scrappiing

In [7]:
import pandas as pd

df1 = pd.read_csv("corrected_classified_odi_players.csv")
df1.head()

,player_id,player,player_role
0,004c9e85,PJ Hughes,Batter
1,008589a4,CM Wright,Unclassified
2,00d69fa4,SO Tikolo,Batter
3,00ea847a,MA Agarwal,Batter
4,00ea9791,HP Tillakaratne,Batter


In [8]:
df2 = pd.read_csv("/home/ai21btech11012/FIFS_dream11/src/data/raw/cricsheet/people.csv")
df2.head()

,identifier,name,unique_name,key_bcci,key_bcci_2,key_bigbash,key_cricbuzz,key_cricheroes,key_crichq,key_cricinfo,...,key_cricingif,key_cricketarchive,key_cricketarchive_2,key_cricketworld,key_nvplay,key_nvplay_2,key_opta,key_opta_2,key_pulse,key_pulse_2
0,b4a23876,AAA Amsterdam,AAA Amsterdam,NaN,NaN,NaN,NaN,NaN,NaN,772407.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,482762af,AA Adeoye,AA Adeoye,NaN,NaN,NaN,NaN,NaN,NaN,380588.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4b0e3049,AA Alleyne,AA Alleyne,NaN,NaN,NaN,NaN,NaN,NaN,661441.0,...,NaN,1512811.0,NaN,NaN,NaN,NaN,NaN,NaN,18542.0,NaN
3,fd3c5c00,AAA Patel,AAA Patel,NaN,NaN,NaN,NaN,NaN,NaN,1163136.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,bc005f0d,AAA White,AAA White,NaN,NaN,NaN,NaN,NaN,NaN,474307.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
import pandas as pd

# Load the datasets
corrected_players = pd.read_csv("corrected_classified_odi_players.csv")
people = pd.read_csv("/home/ai21btech11012/FIFS_dream11/src/data/raw/cricsheet/people.csv")

# Perform the left join
left_joined = pd.merge(
    corrected_players,
    people[["identifier", "key_cricinfo"]],  # Selecting only necessary columns from people.csv
    left_on="player_id",
    right_on="identifier",
    how="left",
)

# Select and reorder the desired columns
final_df = left_joined[["player_id", "player", "player_role", "key_cricinfo"]]

# Display the first few rows of the joined dataframe
print(final_df.head())

# Save the joined dataframe to a new CSV file (optional)
final_df.to_csv("left_joined_players.csv", index=False)


  player_id           player   player_role  key_cricinfo
0  004c9e85        PJ Hughes        Batter      272364.0
1  008589a4        CM Wright  Unclassified       25123.0
2  00d69fa4        SO Tikolo        Batter       24728.0
3  00ea847a       MA Agarwal        Batter      398438.0
4  00ea9791  HP Tillakaratne        Batter       50744.0


In [28]:
df = pd.merge(df, list_df, on='key_cricinfo', how='left')

In [37]:
import re
def categorize_role(row):
    role = row['role']
    player_role = row['player_role']

    if role is None:
        if player_role == 'Batter':
            return 'Batter'
        elif player_role == 'Bowler':
            return 'Bowler'
        elif player_role == 'All-Rounder':
            return 'Allrounder'
        else:
            return 'Unclassified'

    role = str(role).lower()

    if re.search(r'allrounder', role):
        if re.search(r'batting', role):
            return 'Batting Allrounder'
        elif re.search(r'bowling', role):
            return 'Bowling Allrounder'
        else:
            return 'Allrounder'
    elif re.search(r'batter', role):
        return 'Batter'
    elif re.search(r'bowler', role):
        return 'Bowler'
    else:
        return 'Unclassified'

# Apply the function to create the 'final_role' column
df['final_role'] = df.apply(categorize_role, axis=1)

In [53]:
df.to_csv("final_classified.csv", index=False)

## Number of matches played

In [70]:
odi_df = pd.read_csv("/home/ai21btech11012/FIFS_dream11/src/data/interim/ODI_all.csv")
res = odi_df.groupby("player_id")["player_id"].count().reset_index(name="count")
res

,player_id,count
0,004c9e85,25
1,008589a4,7
2,00d69fa4,39
3,00ea847a,5
4,00ea9791,16
...,...,...
1863,ff1ec94d,13
1864,ff3f6fc1,6
1865,ff6a7ab5,35
1866,ffb504b1,26


In [72]:
filtered_res = res[res["count"]>=10]
filtered_res

,player_id,count
0,004c9e85,25
2,00d69fa4,39
4,00ea9791,16
5,012829ff,29
9,0184dc35,126
...,...,...
1860,ff077124,134
1863,ff1ec94d,13
1865,ff6a7ab5,35
1866,ffb504b1,26


In [74]:
merged_df = pd.merge(filtered_res, df, on='player_id', how='left')
merged_df

,player_id,count,player,player_role,key_cricinfo,role,final_role
0,004c9e85,25,PJ Hughes,Batter,272364,Opening Batter,Batter
1,00d69fa4,39,SO Tikolo,Batter,24728,Allrounder,Allrounder
2,00ea9791,16,HP Tillakaratne,Batter,50744,Batter,Batter
3,012829ff,29,JW Hastings,All-Rounder,228622,Allrounder,Allrounder
4,0184dc35,126,R Dravid,Batter,28114,Top order Batter,Batter
...,...,...,...,...,...,...,...
980,ff077124,134,Kamran Akmal,Batter,41028,Wicketkeeper Batter,Batter
981,ff1ec94d,13,R Klein,Unclassified,594996,Bowler,Bowler
982,ff6a7ab5,35,R Trumpelmann,Bowler,698317,Bowler,Bowler
983,ffb504b1,26,XM Marshall,Batter,52626,Opening Batter,Batter


In [75]:
merged_df["role"].isna().sum()

np.int64(9)